## Cutmix 구현 코드
- 기존 이미지에 새로운 이미지를 잘라서 붙이는 방법입니다.
- cutmix를 사용하는 방법은,
- 이미지 데이터셋이 데이터로더를 통과하고 배치단위로 형성된 다음에, 학습에 들어가면서 cutmix를 적용하는 것 같습니다.
- 따라서 부족한 클래스들에 대해서 미리 데이터들을 많이 복사 또는 생성하면 좋을 듯 합니다.  

  
> 2가지 방법이 있습니다.
> 1. 랜덤한 위치에 새로운 이미지를 붙이는 방법 : `def rand_bbox()`
> 2. 가로 또는 세로로 절반을 잘라서 새로운 이미지를 붙이는 방법 : `def half_bbox()`


자세한 내용을 보려면 Cutmix 공식 document를 보면 됩니다.(train.py 확인)    
https://github.com/clovaai/CutMix-PyTorch/tree/2d8eb68faff7fe4962776ad51d175c3b01a25734

In [ ]:
# 1. 랜덤한 위치
def rand_bbox(size, lam): # size : [B, C, W, H]
    W = size[2] # 이미지의 width
    H = size[3] # 이미지의 height
    cut_rat = np.sqrt(1. - lam)  # 패치 크기의 비율 정하기
    cut_w = int(W * cut_rat)  # 패치의 너비
    cut_h = int(H * cut_rat)  # 패치의 높이

    # uniform
    # 기존 이미지의 크기에서 랜덤하게 값을 가져옵니다.(중간 좌표 추출)
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    # 패치 부분에 대한 좌표값을 추출합니다.
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [ ]:
# 2. 이미지의 절반을 자르는 함수
def half_bbox(size, lam):
    W = size[2]
    H = size[3]

    # 가로로 절반(왼쪽, 오른쪽), 세로로 절반(위쪽, 아랫쪽)
    idx = random.randint(0,3)

    bbx1 = [0, W//2, 0, 0]
    bby1 = [0, 0, 0, H//2]
    
    bbx2 = [W//2, W, W, W]
    bby2 = [H, H, H//2, H]

    return bbx1[idx], bby1[idx], bbx2[idx], bby2[idx]
    

저희가 갖고 있는 데이터셋 이름을 `new_df`라 가정하고 코딩을 하면,  
아래 코드 처럼 Dataset과 DataLoader를 먼저 통과를 합니다.

In [ ]:
# cutmix_dataset = CustomDataset(new_df, transforms=data_transform)
# cutmix_dataloader = DataLoader(cutmix_dataset, batch_size=32, shuffle=True)

`lam`(lambda)가 이미지의 어느정도를 자를지를 결정하는 파라미터 입니다.  
이미지를 절반으로 자를 경우, `lam`값이 1/2가 됩니다.

In [ ]:
X, y = next(iter(cutmix_dataloader))

lam = np.random.beta(1.0, 1.0)
rand_index = torch.randperm(X.size()[0]) # 32개, 1개씩 랜덤하게 가져옴
shuffled_y = y[rand_index]

# 2가지 함수 중, 하나를 선택
# bbx1, bby1, bbx2, bby2 = rand_bbox(X.size(), lam)
bbx1, bby1, bbx2, bby2 = half_bbox(X.size(), lam)

# 기존 이미지들의 패치부분을 rand_index를 통해 셔플된 이미지들의 패치로 채움
X[:,:,bbx1:bbx2, bby1:bby2] = X[rand_index,:,bbx1:bbx2, bby1:bby2]

# 다음 iter에 새로운 lambda를 위해 조정
lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (X.size()[-1] * X.size()[-2]))

### Cutmix 결과를 시각화하는 함수

In [ ]:
# cutmix 결과 이미지

def cutmix_plot(train_loader):
    fig , axes = plt.subplots(1,3)
    fig.set_size_inches(15,12)
    
    for i in range(3):
        for inputs, targets in train_loader:
            inputs = inputs
            targets = targets
            break

        lam = np.random.beta(1.0, 1.0) 
        rand_index = torch.randperm(inputs.size()[0])
        
        # bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
        bbx1, bby1, bbx2, bby2 = half_bbox(X.size(), lam)

        inputs[:, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
        lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
        
        axes[i].imshow(inputs[1].permute(1, 2, 0).cpu())
        axes[i].set_title(f'λ : {np.round(lam,3)}')
        axes[i].axis('off')
    return


cutmix_plot(cutmix_dataloader)

### Cutmix를 사용할 때의 Loss function 입니다.  

In [ ]:
# cutmix_criterion : 2개의 정답에 대해 각각의 loss값을 구하여 최종 loss를 반환하는 함
def cutmix_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)